<a href="https://colab.research.google.com/github/daryoush/CudaAndFluxExperiments/blob/main/cudamemoryLeak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Control m m will convert a code cell to a text cell. Control m y will convert a text cell to a code cell.

The full list of keyboard shortcuts is available under 'Keyboard shortcuts' in the Tools menu.

### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.3-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA"]);Pkg.precompile()'
# "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
#    "Random", "DataStructures", "Transformers","Statistics", "BenchmarkTools"\

!echo "DONE"

In [1]:
  [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [10]:
using Pkg
Pkg.add(["BenchmarkTools"])
using CUDA
using Logging

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

if ENV["COLAB_GPU"] == "1"

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


######################################################################### 100.0%


######################################################################### 100.0%


######################################################################### 100.0%


BenchmarkTools.Trial: 
  memory estimate:  704 bytes
  allocs estimate:  38
  --------------
  minimum time:     1.162 ms (0.00% GC)
  median time:      1.499 ms (0.00% GC)
  mean time:        1.506 ms (0.09% GC)
  maximum time:     5.156 ms (28.51% GC)
  --------------
  samples:          3311
  evals/sample:     1

In [4]:
CUDA.memory_status()
GC.gc(true)
CUDA.memory_status()

Effective GPU memory usage: 99.95% (14.719 GiB/14.726 GiB)
CUDA allocator usage: 14.547 GiB
binned usage: 14.516 GiB (32 bytes allocated, 14.516 GiB cached)
Discrepancy of 32.000 MiB between memory pool and allocator!
Effective GPU memory usage: 99.95% (14.719 GiB/14.726 GiB)
CUDA allocator usage: 14.547 GiB
binned usage: 14.531 GiB (16 bytes allocated, 14.531 GiB cached)
Discrepancy of 16.000 MiB between memory pool and allocator!


In [48]:
# Detecting memory leaks

## TODO for some reason the logging doesn't show the debug message!!!
#  The debug messages from memory_status shows the leaks. 
#  figure out how to trun them on in the notebook

CuArray([1])

CUDA.memory_status()

####running Julia on debug level 2 or higher (i.e., with the -g2 argument). When you do so, the memory_status() function from above will display additional information:

Effective GPU memory usage: 1.27% (191.875 MiB/14.726 GiB)
CUDA allocator usage: 16.000 MiB
binned usage: 80 bytes (80 bytes allocated, 0 bytes cached)
Discrepancy of 16.000 MiB between memory pool and allocator!


In [62]:
@debug "debug level log msg, before logger change"
@info "info level msg"
logger = SimpleLogger(stdout, Logging.Debug)
old_logger = global_logger(logger) 
@debug "debug level log msg, after logger change"


┌ Info: info level msg
└ @ Main In[62]:2
